# CHAPTER 39 - DECORATORS

## THE BASICS 

In [94]:
def decorator(cls):
    class Wraper:
        def __init__(self, *args):
            self.wrapper = cls(*args)
        def __getattr__(self, name):
            return getattr(self.wrapper, name)
    return Wraper

In [95]:
@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'spam'

In [96]:
x = C(6,7)

In [97]:
x.attr

'spam'

## CODING FUNCTIONS DECORATORS

In [98]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args):
        self.calls += 1
        print(f'calls {self.calls} to {self.func.__name__}')

In [99]:
@tracer
def spam(a, b, c):
    print(a + b + C)

In [100]:
spam(1, 2, 3)

calls 1 to spam


In [101]:
spam('a', 'b', 'c')

calls 2 to spam


In [102]:
spam.calls

2

## DECORATOR STATE RETENTION OPTIONS

In [103]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print(f'call {self.calls} to {self.func.__name__}')
        return self.func(*args, **kwargs)

In [104]:
@tracer
def spam(a, b, c):
    print(a + b + c)

In [105]:
@tracer
def eggs(x, y):
    print(x ** y)

In [106]:
spam(1, 2, 3)

call 1 to spam
6


In [107]:
spam(a=4, b=5, c=6)

call 2 to spam
15


In [108]:
eggs(2, 16)

call 1 to eggs
65536


In [109]:
eggs(4, y=4)

call 2 to eggs
256


In [110]:
def tracer(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls
        calls += 1
        print(f'call {calls} to {func.__name__}')
        return func(*args, **kwargs)
    return wrapper

In [111]:
@tracer
def spam(a, b, c):
    print(a + b + c)

In [112]:
@tracer
def eggs(x, y):
    print(x ** y)

In [113]:
spam(1, 2, 3)

call 1 to spam
6


In [114]:
spam(a=4, b=5, c=6)

call 2 to spam
15


In [115]:
eggs(2, 16)

call 1 to eggs
65536


In [116]:
eggs(4, y=4)

call 2 to eggs
256


## CLASS BLUNDERS I: DECORATING METODS 

In [117]:
class Person:
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay

    @tracer
    def giveRaise(self, percent):
        self.pay *= (1. + percent)

    @tracer
    def lastName(self):
        return self.name.split()[-1]

In [118]:
bob = Person('Bob Smith', 50000)

In [119]:
bob.giveRaise(.25)

call 1 to giveRaise


In [120]:
print(bob.lastName())

call 1 to lastName
Smith


In [121]:
bob.pay

62500.0

In [122]:
bob.giveRaise(.25)

call 2 to giveRaise


In [123]:
print(bob.lastName())

call 2 to lastName
Smith


## TIMING CALLS 

In [124]:
import time, sys

In [125]:
class timer:
    def __init__(self, func):
        self.func = func
        self.alltime = 0
    def __call__(self, *args, **kargs):
        start = time.time()
        result = self.func(*args, **kargs)
        elapsed = time.time() - start
        self.alltime += elapsed
        print(f'{self.func.__name__}: {self.alltime}')
        return result

In [126]:
@timer
def listcomp(N):
    return [x * 2 for x in range(N)]

In [127]:
@timer
def mapcall(N):
    return list(map((lambda x: x * 2), range(N)))

In [128]:
result = listcomp(5) # Time for this call, all calls, return value
listcomp(50000)
listcomp(500000)
listcomp(1000000)
print(result)
print('allTime = %s' % listcomp.alltime) # Total time for all listcomp calls

listcomp: 4.76837158203125e-06
listcomp: 0.002703428268432617
listcomp: 0.03233218193054199
listcomp: 0.08631587028503418
[0, 2, 4, 6, 8]
allTime = 0.08631587028503418


In [129]:
result = mapcall(5)
mapcall(50000)
mapcall(500000)
mapcall(1000000)
print(result)
print('allTime = %s' % mapcall.alltime)

mapcall: 6.67572021484375e-06
mapcall: 0.006283760070800781
mapcall: 0.07660841941833496
mapcall: 0.21126341819763184
[0, 2, 4, 6, 8]
allTime = 0.21126341819763184


## ADDING DECORATOR ARGUMENTS

In [130]:
def timer(label='', trace=True):
    class Timer:
        def __init__(self, func):
            self.func = func
            self.alltime = 0
        def __call__(self, *args, **kargs):
            import time
            start = time.time()
            result = self.func(*args, *kargs)
            elapsed = time.time() - start
            self.alltime += elapsed
            if trace:
                format = '%s %s: %.5f, %.5f'
                values = (label, self.func.__name__, elapsed, self.alltime)
                print(format % values)
            return result
    return Timer

In [131]:
@timer(label='[CCC]==>')
def listcomp(N):
    return [x * 2 for x in range(N)]

In [132]:
@timer(trace=True, label='[MMM]==>')
def mapcall(N):
    return list(map((lambda x: x * 2), range(N)))

In [133]:
for func in (listcomp, mapcall):
    result = func(5)
    func(50000)
    func(500000)
    func(1000000)
    print(result)
    print('allTime = %s\n' % func.alltime) 
print('**map/comp = %s' % round(mapcall.alltime / listcomp.alltime, 3))

[CCC]==> listcomp: 0.00001, 0.00001
[CCC]==> listcomp: 0.00229, 0.00229
[CCC]==> listcomp: 0.03116, 0.03345
[CCC]==> listcomp: 0.05313, 0.08658
[0, 2, 4, 6, 8]
allTime = 0.08658051490783691

[MMM]==> mapcall: 0.00001, 0.00001
[MMM]==> mapcall: 0.00597, 0.00598
[MMM]==> mapcall: 0.05894, 0.06492
[MMM]==> mapcall: 0.12183, 0.18675
[0, 2, 4, 6, 8]
allTime = 0.18674945831298828

**map/comp = 2.157


## CODING CLASS DECORATORS

In [134]:
instances = {}
def singleton(aClass):
    def onCall(*args, **kwargs):
        if aClass not in instances:
            instances[aClass] = aClass(*args, **kwargs)
        return instances[aClass]
    return onCall

In [135]:
@singleton
class Person:
    def __init__(self, name, hours, rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate

In [136]:
@singleton
class Spam:
    def __init__(self, val):
        self.attr = val    

In [137]:
bob = Person('Bob', 40, 10)
print(bob.name, bob.pay())

Bob 400


In [138]:
sue = Person('Sue', 50, 20)
print(sue.name, sue.pay())

Bob 400


In [139]:
X = Spam(val=42)

In [140]:
Y = Spam(99)

In [141]:
print(X.attr, Y.attr)

42 42


In [142]:
def singleton(aClass):
    instance = None
    def onCall(*args, **kwargs):
        nonlocal instance
        if instance == None:
            instance = aClass(*args, **kwargs)
        return instance
    return onCall

In [143]:
def singleton(aClass):
    def onCall(*args, **kwargs):
        if onCall.instance == None:
            onCall.instance = aClass(*args, **kwargs)
        return onCall.instance
    onCall.instance = None
    return onCall

In [144]:
class singleton:
    def __init__(self, aClass):
        self.aClass = aClass
        self.instance = None
    def __call__(self, *args, **kwargs):
        if self.instance == None:
            self.instance = self.aClass(*args, **kwargs)
        return self.instance

## TRACING OBJECTS INTERFACES

In [145]:
class Wrapper:
    def __init__(self, object):
        self.wrapped = object
    def __getattr__(self, attrname):
        print("Trace:", attrname)
        return getattr(self.wrapped, attrname)

In [146]:
x = Wrapper([1, 2, 3])

In [147]:
x.append(4)

Trace: append


In [148]:
x.wrapped

[1, 2, 3, 4]

In [149]:
x = Wrapper({"a": 1, "b": 2})

In [150]:
list(x.keys())

Trace: keys


['a', 'b']

## Tracing interfaces with class decorators

In [151]:
def Tracer(aClass):
    class Wrapper:
        def __init__(self, *args, **kargs):
            self.fetches = 0
            self.wrapped = aClass(*args, **kargs)
        def __getattr__(self, attrname):
            print('Trace: ' + attrname)
            self.fetches += 1
            return getattr(self.wrapped, attrname)
    return Wrapper


In [152]:
@Tracer
class Spam:
    def display(self):
        print("Spam!" * 8)

In [153]:
@Tracer
class Person:
    def __init__(self, name, hours, rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate

In [154]:
food = Spam()
food.display()
print([food.fetches])

Trace: display
Spam!Spam!Spam!Spam!Spam!Spam!Spam!Spam!
[1]


In [155]:
bob = Person('Bob', 40, 50)
print(bob.name)
print(bob.pay())

Trace: name
Bob
Trace: pay
2000


In [156]:
sue = Person('Sue', rate=100, hours=60)
print(sue.name)
print(sue.pay())

Trace: name
Sue
Trace: pay
6000


In [157]:
print(bob.name)
print(bob.pay())
print([bob.fetches, sue.fetches])

Trace: name
Bob
Trace: pay
2000
[4, 2]


## Class Blunders II: Retaining Multiple Instances

In [158]:
class Tracer:
    def __init__(self, aClass):
        self.aClass = aClass
    def __call__(self, *args):
        self.wrapped = self.aClass(*args)
        return self
    def __getattr__(self, attrname):
        print('Trace: ' + attrname)
        return getattr(self.wrapped, attrname)

In [159]:
@Tracer
class Spam:
    def display(self):
        print('Spam!' * 8)

In [160]:
food = Spam()
food.display()

Trace: display
Spam!Spam!Spam!Spam!Spam!Spam!Spam!Spam!


In [161]:
@Tracer
class Person:
    def __init__(self, name):
        self.name = name

In [162]:
bob = Person('Bob')
print(bob.name)
Sue = Person('Sue')
print(sue.name)
print(bob.name)

Trace: name
Bob
Trace: name
Sue
Trace: name
Sue


## Managing Functions and Classes Directly

In [163]:
registry = {}

In [164]:
def register(obj):
    registry[obj.__name__] = obj
    return obj

In [165]:
@register
def spam(x):
    return (x ** 2)

In [166]:
@register
def ham(x):
    return (x ** 3)

In [167]:
@register
class Eggs:
    def __init__(self, x) -> None:
        self.data = x ** 4
    
    def __str__(self) -> str:
        return str(self.data)

In [168]:
registry


{'spam': <function __main__.spam(x)>,
 'ham': <function __main__.ham(x)>,
 'Eggs': __main__.Eggs}

In [169]:
def decorate(func):
    func.marked = True
    return func

In [170]:
@decorate
def spam(a, b):
    return a + b

In [171]:
spam.marked

True

In [172]:
def annotate(text):
    def decorate(func):
        func.label = text
        return func
    return decorate

In [173]:
@annotate('spam data')
def spam(a, b):
    return a + b

In [174]:
spam(1, 2), spam.label

(3, 'spam data')

In [175]:
"""
Privacy for attributes fetched from class instances.
See self-test code at end of file for a usage example.
Decorator same as: Doubler = Private('data', 'size')(Doubler).
Private returns onDecorator, onDecorator returns onInstance,
and each onInstance instance embeds a Doubler instance.
"""

traceMe = True

def trace(*args):
    if traceMe: print('[' + ' '.join(map(str, args)) + ']')

def Private(*privates):
    def onDecorator(aClass):
        class onInstance:
            def __init__(self, *args, **kargs):
               self.wrapped = aClass(*args, **kargs)

            def __getattr__(self, attr):
                trace('get:', attr)
                if attr in privates:
                    raise TypeError('private attribute fetch: ' + attr)
                else:
                    return getattr(self.wrapped, attr)
                
            def __setattr__(self, attr, value):
                trace('set:', attr, value)
                if attr == 'wrapped':
                    self.__dict__[attr] = value
                elif attr in privates:
                    raise TypeError('private attribute change: ' + attr)
                else:
                    setattr(self.wrapped, attr, value)
        return onInstance
    return onDecorator       
            


In [176]:
@Private('data', 'size')
class Doubler:
    def __init__(self, label, start):
        self.label = label
        self.data = start

    def size(self):
        return len(self.data)

    def double(self):
        for i in range(self.size()):
            self.data[i] = self.data[i] * 2
    
    def display(self):
        print('%s => %s' % (self.label, self.data))

In [177]:
X = Doubler('X is', [1, 2, 3])

[set: wrapped <__main__.Doubler object at 0x7f7a781e1240>]


In [178]:
Y = Doubler('Y is', [-10, -20, -30])

[set: wrapped <__main__.Doubler object at 0x7f7a781e2350>]


In [179]:
X.display()

[get: display]
X is => [1, 2, 3]


In [180]:
X.double()

[get: double]


In [181]:
X.display()

[get: display]
X is => [2, 4, 6]


In [182]:
Y.display()

[get: display]
Y is => [-10, -20, -30]


In [183]:
Y.double()

[get: double]


In [184]:
Y.label = 'Spam'

[set: label Spam]


In [185]:
Y.display()

[get: display]
Spam => [-20, -40, -60]


In [186]:
X.size()

[get: size]


TypeError: private attribute fetch: size

In [ ]:
X.size()

[get: size]


TypeError: private attribute fetch: size

In [ ]:
X.data

[get: data]


TypeError: private attribute fetch: data

In [ ]:
X.data = [1, 1, 1]

[set: data [1, 1, 1]]


TypeError: private attribute change: data

In [ ]:
X.size = lambda S: 0

[set: size <function <lambda> at 0x7f543cd15240>]


TypeError: private attribute change: size

In [ ]:
Y.data

[get: data]


TypeError: private attribute fetch: data

In [ ]:
Y.size()

[get: size]


TypeError: private attribute fetch: size

## Implementation Details I

In [ ]:
traceMe = True
def trace(*args):
    if traceMe: print('[' + ' '.join(map(str, args)) + ']')

def accessControl(failIf):
    def onDecorator(aClass):
        class onInstance:
            def __init__(self, *args, **kargs):
                self.__wrapped = aClass(*args, **kargs)
            
            def __getattr__(self, attr):
                trace('get:', attr)
                if failIf(attr):
                    raise TypeError('private attribute fetch: ' + attr)
                else:
                    return getattr(self.__wrapped, attr)
                
            def __setattr__(self, attr, value):
                trace('set:', attr, value)
                if attr == '_onInstance__wrapped':
                    self.__dict__[attr] = value
                elif failIf(attr):
                    raise TypeError('private attribute change: ' + attr)
                else:
                    setattr(self.__wrapped, attr, value)
        return onInstance
    return onDecorator

def Private(*attributes):
    return accessControl(failIf=(lambda attr: attr in attributes))

def Public(*attributes):
    return accessControl(failIf=(lambda attr: attr not in attributes))

In [ ]:
@Private('age')
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [ ]:
X = Person('Bob', 40)
X.name

[set: _onInstance__wrapped <__main__.Person object at 0x7f544f3b2320>]
[get: name]


'Bob'

In [ ]:
X.name = 'Sue'

[set: name Sue]


In [ ]:
X.name

[get: name]


'Sue'

In [ ]:
X.age

[get: age]


TypeError: private attribute fetch: age

In [ ]:
X.age = 'Tom'

[set: age Tom]


TypeError: private attribute change: age

In [ ]:
@Public('name')
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [ ]:
X = Person('bob', 40)

[set: _onInstance__wrapped <__main__.Person object at 0x7f544f3b15a0>]


In [ ]:
X.name

[get: name]


'bob'

In [ ]:
X.name = 'Sue'

[set: name Sue]


In [ ]:
X.name

[get: name]


'Sue'

In [ ]:
X.age

[get: age]


TypeError: private attribute fetch: age

In [ ]:
X.age = 'Tom'

[set: age Tom]


TypeError: private attribute change: age

## Implementation Details II

In [81]:
"""
File rangetest.py: function decorator that performs range-test
validation for arguments passed to any function or method.
Arguments are specified by keyword to the decorator. In the actual
call, arguments may be passed by position or keyword, and defaults
may be omitted. See rangetest_test.py for example use cases.
"""

trace = True

def rangetest(func):
    def onCall(*pargs, **kargs):
        argchecks = func.__annotations__
        print(argchecks)
        for check in argchecks:
            pass
        return func(*pargs, **kargs)
    return onCall
    # def onDecorator(func):
    #     if not __debug__:
    #         return func
    #     else:
    #         code = func.__code__
    #         allargs = code.co.varnames[:code.co_argcount]
    #         funcname = func.__name__
            
    #         def onCall(*pargs, **kargs):
    #             # All pargs match first N expected args by position
    #             # The rest must be kargs or be omitted default
    #             expected = list(allargs)
    #             positionals = expected[:len(pargs)]

    #             for (argname, (low, high)) in argchecks.items():
    #                 # For all args to be checked
    #                 if argname in kargs:
    #                     #Was passed by name
    #                     if kargs[argname] < low or kargs[argname] > high:
    #                         errmsg = '{0} argument "{1}" not in {2}..{3}'
    #                         errmsg = errmsg.format(funcname, argname, low, high)
    #                         raise TypeError(errmsg)
                        
    #                 elif argname in positionals:
    #                     # Was passed by position
    #                     position = positionals.index(argname)
    #                     if pargs[position] < low or pargs[position] > high:
    #                         errmsg = '{0} argument "{1}" not in {2}..{3}'
    #                         errmsg = errmsg.format(funcname, argname, low, high)
    #                         raise TypeError(errmsg)
    #                 else:
    #                     # Assume not passed: default
    #                     if trace:
    #                         print('Argument "{0}" defaulted'.format(argname))
                    
    #                 return func(*pargs, **kargs) # OK: run original call
    #             return onCall
    #     return onDecorator



In [95]:
@rangetest # persinfo = rangetest(...)(persinfo)
def persinfo(name, age=(0, 120)):
    print('%s is %s years old' % (name, age))

In [83]:
@rangetest
def birthday(M=(1, 12), D=(1, 31), Y=(0, 2013)):
    print('birthday = {0}/{1}/{2}'.format(M, D, Y))

In [84]:
persinfo('Bob', 40)

{}
Bob is 40 years old


In [85]:
persinfo(age=40, name='Bob')

{}
Bob is 40 years old


In [86]:
persinfo(age=150, name='Bob')

{}
Bob is 150 years old


In [87]:
birthday(5, D=1, Y=1963)

{}
birthday = 5/1/1963


## Implementation Details

In [88]:
def func(a, b, c, e=True, f=None):
    x = 1
    y = 2

In [89]:
code = func.__code__

In [90]:
code

<code object func at 0x7fcc1846b050, file "/tmp/ipykernel_2148/3236965857.py", line 1>

In [91]:
code.co_nlocals

7

In [92]:
code.co_varnames

('a', 'b', 'c', 'e', 'f', 'x', 'y')

In [93]:
code.co_argcount

5

In [94]:
code.co_varnames[:code.co_argcount]

('a', 'b', 'c', 'e', 'f')